In [41]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [42]:
##load data:
train_file = '../data/sample_train.csv'
test_file = '../data/test_ver2.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
print train.shape
print test.shape
print 'train_dtypes',train.dtypes
#print 'test_dtypes',test.dtypes

(1000000, 48)
(929615, 24)
train_dtypes fecha_dato                object
ncodpers                   int64
ind_empleado              object
pais_residencia           object
sexo                      object
age                        int64
fecha_alta                object
ind_nuevo                  int64
antiguedad                 int64
indrel                     int64
ult_fec_cli_1t            object
indrel_1mes               object
tiprel_1mes               object
indresi                   object
indext                    object
conyuemp                  object
canal_entrada             object
indfall                   object
tipodom                    int64
cod_prov                 float64
nomprov                   object
ind_actividad_cliente      int64
renta                    float64
segmento                  object
ind_ahor_fin_ult1          int64
ind_aval_fin_ult1          int64
ind_cco_fin_ult1           int64
ind_cder_fin_ult1          int64
ind_cno_fin_ult1           int64
ind

In [43]:
## get the title
fp=open('../data/train_ver2.csv','r')
first_line = fp.readline()
line_title=first_line.strip().split(',')
title = []
for var in line_title:
    title.append(var.strip('\"'))
fp.close()
target = title[24:]
features_object = ['ind_empleado','pais_residencia','sexo',
'ind_nuevo','indrel','indrel_1mes','tiprel_1mes',
'indresi','indext','conyuemp','canal_entrada',
'indfall','tipodom','cod_prov','ind_actividad_cliente',
'segmento']

In [44]:
## Combine into data:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test],ignore_index = True)
## the number of each missing feature
print data.apply(lambda x:sum(x.isnull()))

age                            0
antiguedad                     0
canal_entrada              10232
cod_prov                    8372
conyuemp                 1929387
fecha_alta                     0
fecha_dato                     0
ind_actividad_cliente          0
ind_ahor_fin_ult1         929615
ind_aval_fin_ult1         929615
ind_cco_fin_ult1          929615
ind_cder_fin_ult1         929615
ind_cno_fin_ult1          929615
ind_ctju_fin_ult1         929615
ind_ctma_fin_ult1         929615
ind_ctop_fin_ult1         929615
ind_ctpp_fin_ult1         929615
ind_deco_fin_ult1         929615
ind_dela_fin_ult1         929615
ind_deme_fin_ult1         929615
ind_ecue_fin_ult1         929615
ind_empleado                   0
ind_fond_fin_ult1         929615
ind_hip_fin_ult1          929615
ind_nom_pens_ult1         929615
ind_nomina_ult1           929615
ind_nuevo                      0
ind_plan_fin_ult1         929615
ind_pres_fin_ult1         929615
ind_reca_fin_ult1         929615
ind_recibo

In [45]:
for v in features_object:
    print '\nFrequency count for variable %s'%v
    print data[v].value_counts()


Frequency count for variable ind_empleado
N    1928520
B        453
F        325
A        316
S          1
Name: ind_empleado, dtype: int64

Frequency count for variable pais_residencia
ES    1921246
FR        648
AR        598
DE        598
GB        579
US        488
CO        444
RO        364
IT        364
BR        310
MX        307
VE        283
EC        269
CH        252
BO        191
BE        187
PT        181
PY        171
CL        120
PE        114
CU        103
NL        101
RU         99
PL         87
SE         79
AT         71
CN         69
UY         62
UA         61
BG         59
       ...   
JM          4
BA          4
RS          4
LV          4
LB          4
CF          4
GH          3
IS          3
GI          3
MM          3
GM          3
OM          3
KH          3
CG          3
AL          3
TN          3
TW          3
KW          3
ZW          2
TG          2
BM          2
MT          2
LY          2
BZ          2
DJ          2
KZ          2
SL          1
C

In [46]:
## canal_entrada missing 10232
cannal_entrada_primary = ['KHE','KAT','KFC','KHQ']
data['canal_entrada'] = data['canal_entrada'].apply(lambda x : 'others' if x not in cannal_entrada_primary else x)
print data['canal_entrada'].value_counts()

KHE       516103
KAT       431616
KFC       419267
others    405044
KHQ       157585
Name: canal_entrada, dtype: int64


In [47]:
## cod_prov == nomprov / using cod_prov
cod_prov_primary = [28,8,46,41,15,30,29]
data['cod_prov'] = data['cod_prov'].apply(lambda x : -1 if x not in cod_prov_primary else x)
print data['cod_prov'].value_counts()
data.drop('nomprov',axis = 1,inplace = True)

-1     773978
 28    621062
 8     184668
 46     98602
 41     83970
 15     59410
 30     57318
 29     50607
Name: cod_prov, dtype: int64


In [48]:
## conyuemp
data['conyuemp_missing'] = data['conyuemp'].apply(lambda x:1 if pd.isnull(x) else 0)
print data[['conyuemp','conyuemp_missing']].head(10)
data.drop('conyuemp',axis = 1, inplace = True)
print data['conyuemp_missing'].value_counts()

  conyuemp  conyuemp_missing
0      NaN                 1
1      NaN                 1
2      NaN                 1
3      NaN                 1
4      NaN                 1
5      NaN                 1
6      NaN                 1
7      NaN                 1
8      NaN                 1
9      NaN                 1
1    1929387
0        228
Name: conyuemp_missing, dtype: int64


In [49]:
features_object.remove('conyuemp')
features_object.append('conyuemp_missing')

In [50]:
data['fecha_alta_years'] = data['fecha_alta'].apply(lambda x:int(x[0:4]) - 1990)
print data['fecha_alta_years'].head()
data.drop('fecha_alta', axis = 1, inplace = True)

0    24
1    24
2    24
3    24
4    24
Name: fecha_alta_years, dtype: int64


In [51]:
## fecha_dato
data['fecha_dato_years'] = data['fecha_dato'].apply(lambda x:int(x[0:4])-1990)
data['fecha_years'] = data['fecha_dato_years']-data['fecha_alta_years']
print data['fecha_years'].head()

0    2
1    2
2    2
3    2
4    2
Name: fecha_years, dtype: int64


In [52]:
data.drop(['fecha_dato','fecha_dato_years','fecha_alta_years'],axis = 1,inplace = True)

In [53]:
## ind_empleado N,B,F,A,S -> N,others
data['ind_empleado'] = data['ind_empleado'].apply(lambda x: 'others' if x != 'N' else x)
print data['ind_empleado'].head(20)

0     N
1     N
2     N
3     N
4     N
5     N
6     N
7     N
8     N
9     N
10    N
11    N
12    N
13    N
14    N
15    N
16    N
17    N
18    N
19    N
Name: ind_empleado, dtype: object


In [54]:
data['ind_empleado'].value_counts()

N         1928520
others       1095
Name: ind_empleado, dtype: int64

In [55]:
#indrel_1mes
data['indrel_1mes'].fillna('NAN',inplace = True)
indrel_1mes_features_pnan = ['P','NAN']
data['indrel_1mes'] = data['indrel_1mes'].apply(lambda x: int(float(x)) if x not in indrel_1mes_features_pnan else x)
indrel_1mes_features_test = [1,3,'NAN']
data['indrel_1mes'] = data['indrel_1mes'].apply(lambda x: 'others' if x not in indrel_1mes_features_test else x)

In [56]:
#pais_residencia
data['pais_residencia'] = data['pais_residencia'].apply(lambda x: 'others' if x!= 'ES' else x)
data['pais_residencia'].value_counts()

ES        1921246
others       8369
Name: pais_residencia, dtype: int64

In [57]:
## renta
renta_median = train['renta'].median()
data['renta'] = data['renta'].apply(lambda x: renta_median if str(x).strip()== 'NA' or pd.isnull(x) else float(x))

In [58]:
#segmento
data['segmento'] = data['segmento'].apply(lambda x:'others' if pd.isnull(x)else x)
data['segmento'].value_counts()

02 - PARTICULARES     1131010
03 - UNIVERSITARIO     714137
01 - TOP                73735
others                  10733
Name: segmento, dtype: int64

In [59]:
#sexo
sexo_majority = ['V','H']
data['sexo'].fillna(sexo_majority[random.randint(0,1)],inplace = True)

In [60]:
#tipodom
data.drop('tipodom',axis = 1,inplace = True)

In [61]:
features_object.remove('tipodom')

In [62]:
#tiprel_1mes
tiprel_1mes_majority = ['I','A']
data['tiprel_1mes'].fillna(tiprel_1mes_majority[random.randint(0,1)],inplace = True)

In [63]:
#ult_fec_cli_1t
data.drop('ult_fec_cli_1t',axis = 1,inplace = True)

In [64]:
data.apply(lambda x:sum(x.isnull()))

age                           0
antiguedad                    0
canal_entrada                 0
cod_prov                      0
ind_actividad_cliente         0
ind_ahor_fin_ult1        929615
ind_aval_fin_ult1        929615
ind_cco_fin_ult1         929615
ind_cder_fin_ult1        929615
ind_cno_fin_ult1         929615
ind_ctju_fin_ult1        929615
ind_ctma_fin_ult1        929615
ind_ctop_fin_ult1        929615
ind_ctpp_fin_ult1        929615
ind_deco_fin_ult1        929615
ind_dela_fin_ult1        929615
ind_deme_fin_ult1        929615
ind_ecue_fin_ult1        929615
ind_empleado                  0
ind_fond_fin_ult1        929615
ind_hip_fin_ult1         929615
ind_nom_pens_ult1        929615
ind_nomina_ult1          929615
ind_nuevo                     0
ind_plan_fin_ult1        929615
ind_pres_fin_ult1        929615
ind_reca_fin_ult1        929615
ind_recibo_ult1          929615
ind_tjcr_fin_ult1        929615
ind_valo_fin_ult1        929615
ind_viv_fin_ult1         929615
indext  

In [65]:
## Numerical Coding
le = LabelEncoder()
for col in features_object:
    data[col] = le.fit_transform(data[col])

In [66]:
## One-Hot Coding
data = pd.get_dummies(data, columns=features_object)
data.columns

Index([u'age', u'antiguedad', u'ind_ahor_fin_ult1', u'ind_aval_fin_ult1',
       u'ind_cco_fin_ult1', u'ind_cder_fin_ult1', u'ind_cno_fin_ult1',
       u'ind_ctju_fin_ult1', u'ind_ctma_fin_ult1', u'ind_ctop_fin_ult1',
       u'ind_ctpp_fin_ult1', u'ind_deco_fin_ult1', u'ind_dela_fin_ult1',
       u'ind_deme_fin_ult1', u'ind_ecue_fin_ult1', u'ind_fond_fin_ult1',
       u'ind_hip_fin_ult1', u'ind_nom_pens_ult1', u'ind_nomina_ult1',
       u'ind_plan_fin_ult1', u'ind_pres_fin_ult1', u'ind_reca_fin_ult1',
       u'ind_recibo_ult1', u'ind_tjcr_fin_ult1', u'ind_valo_fin_ult1',
       u'ind_viv_fin_ult1', u'ncodpers', u'renta', u'source', u'fecha_years',
       u'ind_empleado_0', u'ind_empleado_1', u'pais_residencia_0',
       u'pais_residencia_1', u'sexo_0', u'sexo_1', u'ind_nuevo_0',
       u'ind_nuevo_1', u'indrel_0', u'indrel_1', u'indrel_1mes_0',
       u'indrel_1mes_1', u'indrel_1mes_2', u'indrel_1mes_3', u'tiprel_1mes_0',
       u'tiprel_1mes_1', u'tiprel_1mes_2', u'tiprel_1mes_3', u't

In [67]:
## Separate train & test:
train = data.loc[data['source']=='train']
test = data.loc[data['source']=='test']

In [68]:
train.drop('source',axis=1,inplace=True)
test.drop(['source'],axis=1,inplace=True)
test.drop(target,axis = 1,inplace = True)

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [69]:
test.head()

,age,antiguedad,ncodpers,renta,fecha_years,ind_empleado_0,ind_empleado_1,pais_residencia_0,pais_residencia_1,sexo_0,...,cod_prov_6,cod_prov_7,ind_actividad_cliente_0,ind_actividad_cliente_1,segmento_0,segmento_1,segmento_2,segmento_3,conyuemp_missing_0,conyuemp_missing_1
1000000,56,256,15889,326124.90,21,0,1,1,0,0,...,0,0,0,1,1,0,0,0,1,0
1000001,36,34,1170544,101727.81,3,1,0,1,0,1,...,0,0,1,0,0,1,0,0,0,1
1000002,22,34,1170545,101727.81,3,1,0,1,0,0,...,0,0,0,1,0,0,1,0,0,1
1000003,22,34,1170547,148402.98,3,1,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1
1000004,22,34,1170548,106885.80,3,1,0,1,0,1,...,0,0,1,0,0,0,1,0,0,1


In [70]:
train.head()

,age,antiguedad,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,...,cod_prov_6,cod_prov_7,ind_actividad_cliente_0,ind_actividad_cliente_1,segmento_0,segmento_1,segmento_2,segmento_3,conyuemp_missing_0,conyuemp_missing_1
0,51,15,0,0,1,0,0,0,0,0,...,0,1,0,1,1,0,0,0,0,1
1,56,15,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
2,21,15,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
3,21,15,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,1
4,41,15,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1


In [71]:
train.to_csv('../data/train_modified.csv',index=False)
test.to_csv('../data/test_modified.csv',index=False)